In [28]:
from pandas_datareader import data
import datetime
# required modules for Bokeh
from bokeh.plotting import figure, show, output_file
from bokeh.models.annotations import Title


startDate=datetime.datetime(2020,10,2)
endDate=datetime.datetime(2021,1,29)

#object type = DataFrame => store data in a var 'df'
df=data.DataReader(name="BTC-EUR",data_source="yahoo",start=startDate,end=endDate)

def inc_dec(close, open):
    if close > open:
        value="Increase"
    elif close < open:
        value="Decrease"
    else:
        valu="Equal"
    return value
# assign values builds a list in a new Status column
df["Status"]=[inc_dec(close,open) for close, open in zip(df.Close,df.Open)]
# avg between open and close price
df["Middle"]=(df.Open+df.Close)/2
df["Height"]=abs(df.Close-df.Open)

# type(date_increase) # pandas.core.indexes.datetimes.DatetimeIndex

# Build candlestick chart using Bokeh rectangle glyphs
p=figure(x_axis_type='datetime', width=1000, height=300, sizing_mode='stretch_both')
p.title.text = "Candlestick Chart"
p.grid.grid_line_alpha=0.3  # horizontal grid line transparency

# Hours to miliseconds
hours_12=12*60*60*1000

# four mandatory params for each candle:
# 1. x-coord of rect centre point   2. y-coord of rect centre point
# 3. width of rect   4. height of rect

# Plot high, low line behind candle stick
p.segment(df.index, df.High, df.index, df.Low, color="black")

# Plot candle sticks where close is gt than open - stock price is up for session
p.rect(df.index[df.Status=="Increase"],df.Middle[df.Status=="Increase"], 
      hours_12, df.Height[df.Status=="Increase"], fill_color="#00ff40", line_color="black") # abs ensures a positive value always

# Plot candle sticks where close is lt open - stock price is down for session
p.rect(df.index[df.Status=="Decrease"],df.Middle[df.Status=="Decrease"], hours_12, 
       df.Height[df.Status=="Decrease"], fill_color="#ff4d4d", line_color="black") # abs ensures a positive value always


output_file("CS.html")
show(p)
